In [1]:
from sifce import datatools, population, network
import pandas as pd
import numpy as np
from bilby.core.prior import Uniform, Sine, Cosine, PowerLaw, Constraint, DeltaFunction
from pycbc.detector import Detector
from bilby.core.prior import PriorDict
from pycbc.psd import analytical
pd.set_option('display.max_columns', None)

In [2]:
net = network.Network(ifos=["H1", "L1"])

In [3]:
sample_rate=1024
flow = 20.0
delta_f = 1.0/8
flen = int(sample_rate/delta_f)
psd = analytical.aLIGOZeroDetHighPower(flen, delta_f, flow)

net.network_psds = {'H1': psd, 'L1': psd, 'V1':psd}

In [4]:
prior_gw150914 = dict(mass_1= DeltaFunction(35),
                     mass_2= DeltaFunction(35),
                     a_1 = DeltaFunction(0.5),
                     a_2 = DeltaFunction(0.5),
                     tilt_1 = DeltaFunction(1.4),
                     tilt_2 = DeltaFunction(1.4),
                     phi_12 = DeltaFunction(3.09),
                     phi_jl = DeltaFunction(5.21),
                     theta_jn =  Sine(name='theta_jn'),
                     phase = DeltaFunction(1.04),
                     reference_frequency = DeltaFunction(20),
                     dec=Cosine(name='dec'),
                     ra=Uniform(name='ra', minimum=0, maximum=2 * np.pi, boundary='periodic'),
                     psi =  Uniform(name='psi', minimum=0, maximum=np.pi, boundary='periodic'),
                     luminosity_distance = PowerLaw(alpha=2, name='luminosity_distance', minimum=50, maximum=2000, unit='Mpc', latex_label='$d_L$'))

In [5]:
prior_obj = PriorDict(dictionary = prior_gw150914)
pop = population.SimulationSet(distribution=prior_obj)
pop.sample_distribution(100)

In [6]:
for i in range(len(pop.simulations_dataframe.index)):
    row = pop.simulations_dataframe.iloc[[0]]
    pop.calc_snrs_sky(row, net, N=10)

In [ ]:
for i in range(len(pop.simulations_dataframe.index)):
    row = pop.simulations_dataframe.iloc[[0]]
    pop.sample_distances(row, net, N=10)

In [ ]:
pop.simulations_dataframe

In [ ]:
from astropy.cosmology import Planck18
tv = pop.sensitive_volume_calc(11, Planck18)

In [ ]:
tv